# Форматирование стандартных таблиц из БД 

In [ ]:
# --- Форматирование таблиц для отчётности --- 


# ---
# Сбрасываем timezones дат из подгруженных таблиц
tables_for_report = [
    user_session,
    users, 
    user_info, 
    virtual_accounts, 
    loans, 
    loan_payments,
    payments,
    operations,
    score,
    equifax_entries,
    transactions,
    reviews,
    loan_investors,
    loan_investors_payments
]


# Цикл сброса timezones
for table in tables_for_report: 
    main_module.functions.time_functions.drop_tz(table)

In [ ]:
# Модификация loans (убираем тестовые заявки) 


# ---
# Выделяем тестовые заявки по двум параметрам: is_test, is_tester == True
test_users_ids = users[((users.is_test == True) | (users.is_tester == True)) & (users.id != 32649)].id

# Фильтр по тестовым users
users = users[~users.id.isin(test_users_ids)]

# Фильтр по тестовым user_info
user_info = user_info[~user_info.user_id.isin(test_users_ids)]

# Фильтр по тестовым user_session
user_session = user_session[~user_session.user_id.isin(test_users_ids)]

# Фильтр по тестовым virtual_accounts
virtual_accounts = virtual_accounts[~virtual_accounts.user_id.isin(test_users_ids)]

# Фильтр по тестовым loans
loans = loans[~((loans.is_test == True)
              | (loans.owner_id.isin(test_users_ids))
              | (loans.debtor_id.isin(test_users_ids))
              | (loans.investor_id.isin(test_users_ids)))]

# Фильтр по тестовым loan_payments
loan_payments = loan_payments[~((loan_payments.debtor_id.isin(test_users_ids)) | 
                              (loan_payments.investor_id.isin(test_users_ids)))]

# Фильтр по тестовым payments
payments = payments[~((payments.from_id.isin(test_users_ids))
                    | (payments.to_id.isin(test_users_ids)))]

# Фильтр по тестовым operations
operations = operations[(operations.virtual_account_id.isin(virtual_accounts.id))]

# Фильтр по тестовым transactions
transactions = transactions[~transactions.user_id.isin(test_users_ids)]

# Фильтр по тестовым score
score = score[~score.user_id.isin(test_users_ids)]



# ---
# Изменение таблицы, перенос заявок из группы F2F в группу P2P для ID из списка 
loans.loc[(loans.id.isin([22072, 22494, 23208, 23239, 23357, 24017 ,23999, 24188, 24164, 24320, 
                          24251, 24241, 24424, 24402, 24388, 24435, 24955, 25926, 26131, 26125, 
                          26556, 27441, 27431, 27852, 27997, 28270, 28544, 28573, 30503, 30865, 
                          30862, 30856, 30847, 30837, 30952, 31019, 30999, 31167, 31143, 31225, 
                          31196, 31185, 31427, 31775, 31757, 31827, 31807, 31793, 31786, 32015, 
                          32083, 32150, 32174, 32484, 32583, 32560, 32533, 32527, 32518, 32590, 
                          32893, 33003, 33065, 33204, 33307, 33456, 33400, 33605, 33523, 33679,
                          33700, 33909, 33866, 33852])), ['is_collective']] = True



# ---
# Сортировка таблицы с займами
loans = loans.sort_values(by='id', ascending=False)

# Генерация новых таблиц для отчётов

In [ ]:
# Выделение p2p, f2f и old_debt таблиц

# --- 
# Фильтр по P2P заявкам (полный) - ТОЛЬКО ВСЕ P2P ЗАЯВКИ
loans_p2p = loans[loans.is_collective == True]

# Удаление заявок 1 310 000
loans_p2p = loans_p2p.drop(loans_p2p[loans_p2p.id.isin([56929, 56893, 56889, 56886, 56841, 56794, 56728, 56723, 56718,
                                                        56552, 56546, 56304, 56267, 56246, 56242, 56202, 56185, 56147,
                                                        56039, 56032, 56031, 56009, 55964, 55936, 55925, 55817, 55813,
                                                        55766, 55757, 55662, 55529, 55457, 55442, 55312, 55287, 55267,
                                                        55254, 55188, 55061, 54961, 54955, 54893, 54829, 54776, 54750,
                                                        54717, 54694, 54693, 54675, 54660])].index)

# --- 
# Фильтр по F2F заявкам (полный) - ТОЛЬКО ВСЕ F2F ЗАЯВКИ
loans_f2f = loans[(loans.type == 'new_debt')
                  & ((loans.is_collective == False) | (loans.is_collective.isna()))]

# --- 
# Фильтр по LT заявкам (полный) - ТОЛЬКО ВСЕ LT ЗАЯВКИ
loans_lt = loans[loans.type == 'old_debt']

In [ ]:
# Выделение таблиц для отчётности
user_session_report = user_session
users_report, user_info_report, virtual_accounts_report = users, user_info, virtual_accounts
loans_report, loans_p2p_report, loans_f2f_report = loans, loans_p2p, loans_f2f
loan_payments_report, payments_report, operations_report = loan_payments, payments, operations
score_report, equifax_entries_report = score, equifax_entries

In [ ]:
# Создание таблицы investments, как loans но со стороны инвестора
kws = ['date_updated','next_payment_date','investor_id_y','type','contract_required','remind_day','owner_id',
       'investor_card_for_payments_id','debtor_card_for_payments_id','reminded_date','is_cash','id_y','endless']

investments = loan_investors.merge(loans,
                                   left_on = 'loan_id',
                                   right_on = 'id',
                                   how = 'left').drop(kws, axis=1).iloc[:,0:21]


investments['commision'] = loan_investors.merge(loans,
                                                left_on = 'loan_id',
                                                right_on = 'id',
                                                how = 'left').drop(kws, axis=1).iloc[:,35]

investments['amount_x'] = investments['amount_x'] - investments['amount_x'] / (investments['sum'] + investments['commision']) * investments['commision']
investments['total_sum'] *= investments['amount_x'] / investments['sum']
investments['month_payment'] *= investments['amount_x'] / investments['sum']
investments['amount_left'] *= investments['amount_x'] / investments['sum']
investments['amount_paid'] *= investments['amount_x'] / investments['sum']
investments['commision'] *= investments['amount_x'] / investments['sum']
investments = investments.rename(columns={'id_x':'id','investor_id_x':'investor_id','amount_x':'amount'}).drop('amount_y',axis=1)

# Добавление нормализованных столбцов

In [ ]:
# Сумма займа с учётом комиссии
loans_p2p['sum_with_comm'] = loans_p2p['sum'] + loans_p2p['commission']

In [ ]:
# Добавляем нормализованную длительность займа в месяцах
loans['duration_month'] = loans[['period',
                                 'duration']].apply(main_module.functions.time_functions.normalize_duration, axis=1)
loans_p2p['duration_month'] = loans_p2p[['period',
                                         'duration']].apply(main_module.functions.time_functions.normalize_duration, axis=1)
investments['duration_month'] = investments[['period',
                                             'duration']].apply(main_module.functions.time_functions.normalize_duration, axis=1)

In [ ]:
# Перевод ставки по займам в % годовых
loans_p2p_report.loc[(loans_p2p_report['rate_type'] == 'yearly'), 'rate_yearly'] = loans_p2p_report['rate'] 
loans_p2p_report.loc[(loans_p2p_report['rate_type'] == 'monthly'), 'rate_yearly'] = 12 * loans_p2p_report['rate']
loans_p2p_report.loc[(loans_p2p_report['rate_type'] == 'daily'), 'rate_yearly'] = 365 * loans_p2p_report['rate']